# Errors and Exceptions

## 1. The Try-Except Construct

Along our journey learning Python, we've encountered errors generated by the interpreter a bunch of times. We've seen examples of TypeError, IndexError, ValueError, and others. Up to now whenever the interpreter threw one of these errors we changed our code to avoid the error. That's a common approach since whenever the interpreter raises one of these errors the program stops, and we don't want our scripts to come to an end before they're done doing their work. Sometimes it's easier to make a verification with the conditional to avoid the error. Like in our earlier example of the rearranged name function where we check if the result from our regular expression search was none and did something different in that case. 

Other times there are so many things that could go wrong that checking for all of them becomes challenging. Say you had a function that opened a file and did some processing on it. What if the file doesn't exist? What if the user doesn't have permissions to read the file? Or what if the file is locked by different process and can't be opened right now? We could check all of these conditions but what if there's yet another thing that could cause the open function to raise an error. In a case like this, a better approach is to use the try-except construct. Let's look at how it works in an example. 


```python
def char_frequency(filename):
    try:
        file = open(filename)
    except OSError:
        return none
    
    ...
```
Our character_frequency function here reads the contents of a file to count the frequency of each character in them. To do that, the first step is to open the file. In this example, we've put the call to the open function inside a try-except block. What this does is first try to do the operation that we want which in this case is to open the file. If there's an error, it then goes into the accept part of the block that matches the error and does whatever cleanup is necessary.

Here we have only one except block, for the `OSError` error type, but there could be more blocks if the functions called could raise other types of errors. So when writing a try-except block, the important thing to remember is that **the code in the except block is only executed if one of the instructions in the try block raise an error of the matching type**. 

In this case, in the except-block, we're returning none to indicate to the calling code that the function wasn't able to do what was requested of it. Returning none when something fails is a common pattern but not the only one. 

We could also decide to set a variable to some base value like zero for numbers, empty string for strings, empty list for list, and so on. It all depends on what our function does and what we need to get that work done. The important point is that when we have an operation that might raise an error we want handle that failure gracefully by using the try-except block. The operation could be opening a file, converting a value to a different format, executing a system command, sending data over the network or any other action that might fail and isn't trivial to check with a conditional. 

To use a try-except block, we need to be aware of the errors that functions that we're calling might raise. This information is usually part of the documentation of the functions. Once we know this we can put the operations that might raise errors as part of the try block, and the actions to take when errors are raised as part of a corresponding except block. You're probably asking yourself, how do I raise my own errors? Lucky for you that's up next. We'll dive into how to raise our own errors when necessary.

## 2. Raising Errors

In the last video, we looked into how to handle errors when they're raised by the functions that we call. In some cases, we might want to raise an error ourselves. This usually happens when some of the conditions necessary for a function to do its job properly aren't met and returning none or some other base value isn't good enough. Let's look at this through an example. 

Say we had a function that verifies whether a chosen username is valid. One of the checks this function does is verify that the provided name is at least a certain amount of characters with the minimum value received by a parameter. Something like this.

In [1]:
def validate_user(username, minlen):
    if len(username) < minlen:
        return False
    if not username.isalnum():
        return False
    return True

In this function, we're first checking that the username variable has at least minlen characters. After checking that, we verify if there are any non-alphanumeric characters in the string which is another criteria for validating a username. If all the checks pass we return true to indicate that the username chosen is valid. 

This code works as long as the provided values are sensible. What would happen if the minlen variable is zero or negative number? Our function will allow an empty username as valid which doesn't make much sense. 

To prevent this from happening, we can add an extra check to our function which will verify the receipt parameters are sane. In this case, returning false would be misleading because it's not necessarily that the username is invalid but the provided minlen value doesn't make sense. So let's add a check to verify that minlen is at least one and raise an error if that's not the case.

In [2]:
def validate_user(username, minlen):
    if minlen < 1:
        raise ValueError('minlen must be at least 1')
    if len(username) < minlen:
        return False
    if not username.isalnum():
        return False
    return True

Cool. As you can see, the keyword to generate an error in Python is `raise`. We can raise a bunch of different errors that come already pre-built with Python or we can create our own, if the standard ones aren't good enough. In this case, we're raising a `ValueError`, a type of error that we've come across before to indicate that there was a problem with one of the values of the parameters. Let's save our code and then try it out in the interpreter.

In [4]:
validate_user('MrKrabsWarCriminal', -1)

ValueError: minlen must be at least 1

Success. We imported our function and called it with an invalid parameter. Our function successfully raised an error just like we wanted. I bet you didn't expect that there'll be a point where getting an error would mean you're doing things right, ha? Let's also try calling it with valid parameters to see if those work.

In [5]:
validate_user('MrKrabsWarCriminal', 2)

True

In [6]:
validate_user(3, 2)

TypeError: object of type 'int' has no len()

In this case, the Python interpreter raised an error because our code is trying to use the length function and we can't do that with integers. Let's start passing a list which does have a len function. First, an empty list.

In [7]:
validate_user([], 2)

False

Because this list is shorter than the minimum length, our code returned false. Now, let's try it with a list of one element.

In [10]:
# NOTE: so apparently this works when it's not supposed to
validate_user(['name'], 2)

False

In [11]:
validate_user(['name', 'test'], 2)

AttributeError: 'list' object has no attribute 'isalnum'

So in this example, we got a different error because we were trying to use the `isalnum` method which is not available on list. We managed to get three different possible results when passing a value that wasn't a string. 

Depending on how our function is going to be used, this could be okay. It's usually the responsibility of whoever is calling a function to call it the right parameters. But in some cases, we might want to do this explicitly by checking that we're receiving a value that makes sense to that function. So let's look at an alternative to the raise keyword that we can use for situations where we want to check that our code behaves the way it should particularly when we want to avoid situations that should never happen. 

This is the `assert` keyword. This keyword tries to verify that a conditional expression is true, and if it's false it raises an assertion error with the indicated message. Let's add an assertion to our function.

In [12]:
def validate_user(username, minlen):
    assert type(username) == str, 'username must be string'
    if minlen < 1:
        raise ValueError('minlen must be at least 1')
    if len(username) < minlen:
        return False
    if not username.isalnum():
        return False
    return True

We've added an assertion that verifies that the type of the username variable is STR which we know is a name that the interpreter uses for strings. If the function is called with a username parameter that's not a string, an error will be raised with the message we provided. Let's try this out. First, we'll need to close our interpreter and restart it to import the modified module.

In [13]:
validate_user([3], 4)

AssertionError: username must be string

We see that our function now raises an error type assertion error if the first parameter isn't a string. As we've called out, we usually don't need to check the types of our parameters. Depending on what our function does, it might be perfectly okay for it to allow scripts to call it with parameters of different types. Assertions can be super helpful for debugging some code that's not behaving the way we expect it to. We can add them at any point where we want to ensure that the variables contain the values and types that they should or when we think that's something that shouldn't happen is happening. Heads up though. Assertions will get removed from our code if we ask the interpreter to optimize it to run faster. So as a rule, we should use raise to check for conditions that we expect to happen during normal execution of our code and assert to verify situations that aren't expected but that might cause our code to misbehave. 

By now, we've seen how we can handle errors when the code we call generates them and how we can raise our own errors when we want our code to signal that something hasn't gone well. This is complex stuff and it's okay if it takes a little while for it to sink in. As usual, we'll include a cheat sheet and give you plenty of opportunities to practice your newly acquired error handling skills. Up next, we'll look into how we can add test to verify that a function raises the errors that it needs to raise.